In [1]:
# Setup logging
from timeseria import logger
logger.setup(level='INFO')

⚠️ This Notebook renders with limited features (e.g. no plots) if opened on GitHub, or as an untrusted Notebook in your local environment. Please either open it using [NBViewer](https://nbviewer.org/github/sarusso/Timeseria-notebooks/tree/master/notebooks/) or mark it as [trusted](https://stackoverflow.com/questions/44943646/jupyter-notebook-not-trusted).


# Temperature and humidity forecasting with LSTM


This notebook showcase the periodic average forecatser of [Timeseria](https://github.com/sarusso/Timeseria).

This forecaster is very simple (yet somehow powerful) but most importanlty it fits nearly instantaneously and it has the very same interfaces as its more powerful counterparts as the neural network-based ones.

It is a very useful "toy forecaster" to get used with a number of things including data windowing and model accuracy evaluation.

In [2]:
from timeseria import  storages

# Set sataset path to Timeria test CSV data
DATASET_PATH = '/'.join(storages.__file__.split('/')[0:-1]) + '/tests/test_data/csv/'

# Create a CSV storage pointing to a temperature and humidity dataset
csv_storage = storages.CSVFileStorage(DATASET_PATH + 'humitemp_long.csv')

# Get the time series out from the CSV file storage
timeseries = csv_storage.get()

In [3]:
# Have a look at the time series
timeseries

Time series of #17910 points at variable resolution (~615s), from point @ 1546475294.0 (2019-01-03 00:28:14+00:00) to point @ 1557964581.0 (2019-05-15 23:56:21+00:00)

In [4]:
# Plot the time series
timeseries.plot()

[INFO] timeseria.plots: Aggregating by "10" for improved plotting


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
# Resample to one hour, and make data uniform
timeseries = timeseries.resample('1h')

[INFO] timeseria.transformations: Using auto-detected sampling interval: 615.0s
[INFO] timeseria.transformations: Resampled 17910 DataTimePoints in 3191 DataTimePoints


## Model fit, usage and evaluation

In [6]:
from timeseria.models import LSTMForecaster

# Instantiate and fit the model
forecaster = LSTMForecaster(window=24, neurons=256, features=['values', 'diffs', 'hours'])
forecaster.fit(timeseries, epochs=5, verbose=True)

Epoch 1/5
99/99 [==============================] - 3s 21ms/step - loss: 0.0092
Epoch 2/5
99/99 [==============================] - 2s 22ms/step - loss: 0.0021
Epoch 3/5
99/99 [==============================] - 2s 21ms/step - loss: 0.0018
Epoch 4/5
99/99 [==============================] - 2s 23ms/step - loss: 0.0017
Epoch 5/5
99/99 [==============================] - 2s 21ms/step - loss: 0.0017


In [7]:
# Call the predict() function of the model. This returns key-value data with the prediction values
forecaster.predict(timeseries)

{'humidity': 40.846313795257075, 'temperature': 22.014964171476226}

In [8]:
# The timeseries on which to call the predict has to be at least of 24 hours (the window length)
try:
    forecaster.predict(timeseries[0:23])
except Exception as e:
    print(e)

The timeseries length (23) is shorter than the model window (24), it must be at least equal.


In [9]:
# The forecats() method returns not only data but also their data points, with the timestamp
forecaster.forecast(timeseries)

Time point @ 1557964800.0 (2019-05-16 00:00:00+00:00) with data "{'humidity': 40.846313795257075, 'temperature': 22.014964171476226}"

In [10]:
# The apply() method allows instead to directly apply the model on a time series
# (by combining the forecasts with the real observations).
timeseries_with_forecast = forecaster.apply(timeseries, steps=24)

In [11]:
# Plot the time series together with the prediction
timeseries_with_forecast.plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>